In [2]:
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

from utils import GLOBAL, functions

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [4]:
data_df = pd.DataFrame()

yearStart = 2014
yearEnd = 2021

WEEKLY_BASE_URL = "https://raw.githubusercontent.com/fantasydatapros/data/master/weekly/{year}/week{week}.csv"

for year in range(yearStart, yearEnd):
    for week in range(1, 18):
        weekly_df = pd.read_csv(WEEKLY_BASE_URL.format(year=year, week=week))
        weekly_df['Year'] = year
        weekly_df['Week'] = week
        weekly_df['Usage'] = (weekly_df['PassingAtt'] + weekly_df['RushingAtt'] + weekly_df['Tgt'])
        data_df = pd.concat([data_df, weekly_df])

In [5]:
#Cleaning up TM values
data_df.loc[(data_df['Tm'] == 'OTI'), 'Tm'] = 'TEN'
data_df.loc[(data_df['Tm'] == 'RAM'), 'Tm'] = 'LAR'
data_df.loc[(data_df['Tm'] == 'STL'), 'Tm'] = 'LAR'
data_df.loc[(data_df['Tm'] == 'HTX'), 'Tm'] = 'HOU'
data_df.loc[(data_df['Tm'] == 'SDG'), 'Tm'] = 'LAC'
data_df.loc[(data_df['Tm'] == 'OAK'), 'Tm'] = 'LV'
data_df.loc[(data_df['Tm'] == 'RAI'), 'Tm'] = 'LV'
data_df.loc[(data_df['Tm'] == 'CLT'), 'Tm'] = 'IND'
data_df.loc[(data_df['Tm'] == 'RAV'), 'Tm'] = 'BAL'
data_df.loc[(data_df['Tm'] == 'CRD'), 'Tm'] = 'ARI'
data_df.loc[(data_df['Tm'] == 'NOR'), 'Tm'] = 'NO'

In [6]:
data_df = data_df.groupby(['Player', 'Pos', 'Tm', 'Year'], as_index=False)\
    .agg({
        'Usage': np.sum,
        'PassingYds': np.sum,
        'PassingTD': np.sum,
        'PassingAtt': np.sum,
        'RushingAtt': np.sum,
        'RushingYds': np.sum,
        'RushingTD': np.sum,
        'Rec': np.sum,
        'Tgt': np.sum,
        'ReceivingYds': np.sum,
        'ReceivingTD': np.sum,
        'PPRFantasyPoints': np.sum,
        'StandardFantasyPoints': np.sum,
        'HalfPPRFantasyPoints': np.sum
    })

In [7]:
#Set Scoring format that will be used below ('HalfPPR', 'PPR', or 'Standard')
scoring_format = 'HalfPPR'

In [8]:
pd.set_option('chained_assignment', None)

lag_features = [
    'RushingAtt',
    'Tgt',
    'Usage', 
    f'{scoring_format}FantasyPoints', 
    'PassingAtt', 
    'PassingTD'
]

for lag in range(1, 7):
    shifted = data_df.groupby('Player').shift(lag)

    for column in lag_features:
        data_df[f'lag_{column}_{lag}'] = shifted[column]
        
data_df = data_df.fillna(-1)

In [9]:
#Separate by pos
wr_df = data_df.loc[data_df['Pos'] == 'WR']
rb_df = data_df.loc[data_df['Pos'] == 'RB']
te_df = data_df.loc[data_df['Pos'] == 'TE']
qb_df = data_df.loc[data_df['Pos'] == 'QB']

WRs

In [27]:
X = wr_df[['lag_Tgt_1', 'lag_RushingAtt_1', 'lag_PassingAtt_1', 'lag_Usage_1', f'lag_{scoring_format}FantasyPoints_1']]
y = wr_df[f'{scoring_format}FantasyPoints'].values

WR_X_train, WR_X_test, WR_y_train, WR_y_test = train_test_split(X, y, test_size=0.2, random_state=10)

lr = LinearRegression()

lr.fit(WR_X_train, WR_y_train)

WR_y_predict = lr.predict(WR_X_test)

mean_absolute_error(WR_y_test, WR_y_predict)

40.934881555892886

In [ ]:
wr_df_pred = wr_df.loc[
    (wr_df['Usage'] > 50) & (wr_df['Year'] == 2020),
     ['Player', 'Tgt', 'RushingAtt', 'PassingAtt', 'Usage', f'{scoring_format}FantasyPoints']
]

wr_df_pred['Predicted_2021'] = lr.predict(
    wr_df_pred[['Tgt', 'RushingAtt', 'PassingAtt', 'Usage', f'{scoring_format}FantasyPoints']].values
)

wr_df_pred.sort_values(by='Predicted_2021', ascending=False).head(100)

RBs

In [29]:
X = rb_df[['lag_Tgt_1', 'lag_RushingAtt_1', 'lag_PassingAtt_1', 'lag_Usage_1', f'lag_{scoring_format}FantasyPoints_1']]
y = rb_df[f'{scoring_format}FantasyPoints'].values

RB_X_train, RB_X_test, RB_y_train, RB_y_test = train_test_split(X, y, test_size=0.2, random_state=10)

lr = LinearRegression()

lr.fit(RB_X_train, RB_y_train)

RB_y_predict = lr.predict(RB_X_test)

mean_absolute_error(RB_y_test, RB_y_predict)

50.844107901831975

In [ ]:
rb_df_pred = rb_df.loc[
    (rb_df['Usage'] > 50) & (rb_df['Year'] == 2020),
     ['Player', 'Tgt', 'RushingAtt', 'PassingAtt', 'Usage', f'{scoring_format}FantasyPoints']
]

rb_df_pred['Predicted_2021'] = lr.predict(
    rb_df_pred[['Tgt', 'RushingAtt', 'PassingAtt', 'Usage', f'{scoring_format}FantasyPoints']].values
)

rb_df_pred.sort_values(by='Predicted_2021', ascending=False).head(100)

TEs

In [31]:
X = te_df[['lag_Tgt_1', 'lag_RushingAtt_1', 'lag_PassingAtt_1', 'lag_Usage_1', f'lag_{scoring_format}FantasyPoints_1']]
y = te_df[f'{scoring_format}FantasyPoints'].values

TE_X_train, TE_X_test, TE_y_train, TE_y_test = train_test_split(X, y, test_size=0.2, random_state=10)

lr = LinearRegression()

lr.fit(TE_X_train, TE_y_train)

TE_y_predict = lr.predict(TE_X_test)

mean_absolute_error(TE_y_test, TE_y_predict)

30.509183392063107

In [ ]:
te_df_pred = te_df.loc[
    (te_df['Usage'] > 50) & (te_df['Year'] == 2020),
     ['Player', 'Tgt', 'RushingAtt', 'PassingAtt', 'Usage', f'{scoring_format}FantasyPoints']
]

te_df_pred['Predicted_2021'] = lr.predict(
    te_df_pred[['Tgt', 'RushingAtt', 'PassingAtt', 'Usage', f'{scoring_format}FantasyPoints']].values
)

te_df_pred.sort_values(by='Predicted_2021', ascending=False).head(100)

QBs

In [33]:
X = qb_df[['lag_Tgt_1', 'lag_RushingAtt_1', 'lag_PassingAtt_1', 'lag_Usage_1', f'lag_{scoring_format}FantasyPoints_1']]
y = qb_df[f'{scoring_format}FantasyPoints'].values

QB_X_train, QB_X_test, QB_y_train, QB_y_test = train_test_split(X, y, test_size=0.2, random_state=10)

lr = LinearRegression()

lr.fit(QB_X_train, QB_y_train)

QB_y_predict = lr.predict(QB_X_test)

mean_absolute_error(QB_y_test, QB_y_predict)

64.04875933809929

In [34]:
qb_df_pred = qb_df.loc[
    (qb_df['Usage'] > 50) & (qb_df['Year'] == 2020),
     ['Player', 'Tgt', 'RushingAtt', 'PassingAtt', 'Usage', f'{scoring_format}FantasyPoints']
]

qb_df_pred['Predicted_2021'] = lr.predict(
    qb_df_pred[['Tgt', 'RushingAtt', 'PassingAtt', 'Usage', f'{scoring_format}FantasyPoints']].values
)

qb_df_pred.sort_values(by='Predicted_2021', ascending=False).head(100)

,Player,Tgt,RushingAtt,PassingAtt,Usage,HalfPPRFantasyPoints,Predicted_2021
52,Aaron Rodgers,1.0,38.0,526.0,565.0,382.76,327.760404
3823,Kyler Murray,0.0,133.0,558.0,691.0,378.74,326.568468
1901,Deshaun Watson,0.0,90.0,544.0,634.0,367.32,315.418874
5159,Russell Wilson,0.0,83.0,558.0,641.0,357.78,304.341270
5238,Ryan Tannehill,1.0,43.0,481.0,525.0,337.78,292.859128
4755,Patrick Mahomes,0.0,61.0,543.0,604.0,336.92,285.909844
3844,Lamar Jackson,0.0,137.0,325.0,462.0,298.00,284.280253
3337,Josh Allen,0.0,80.0,458.0,538.0,317.60,280.834664
5699,Tom Brady,0.0,30.0,610.0,640.0,337.92,275.889083
3479,Justin Herbert,0.0,55.0,595.0,650.0,328.84,271.075985
